In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import yfinance as yf

In [ ]:
user = "root"
password = "fBTJ3_a_3Rg^V!K"
host = "localhost"     # or server IP
database = "stockprice_db"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

def sql_push(df,table_name):
    df.to_sql(name = table_name, con = engine, if_exists = "replace", index = False)

def download_push(ticker, tab_name):
    sdf = yf.download(ticker, start= "2015-04-01")
    sdf.reset_index(inplace=True)
    sdf = pd.DataFrame(sdf)
    sql_push(sdf, tab_name)

In [34]:
download_push(ticker="IOC.NS", tab_name="iocl")

[*********************100%***********************]  1 of 1 completed


In [38]:
input("Run the query in 'ioc table prep.sql' file in MySQL, once done Press Enter to Continue ")
log_df = pd.read_sql("SELECT date, close, log_close FROM iocl_clean", engine)
print(log_df.head())

         date  close  log_close
0  2015-11-24  37.59       3.63
1  2015-11-25  37.59       3.63
2  2015-11-26  37.82       3.63
3  2015-11-27  37.68       3.63
4  2015-11-28  37.68       3.63


In [39]:
def index_date(df):
    rdf = df.copy()
    rdf['date'] = pd.to_datetime(df['date'])
    rdf.set_index('date', inplace = True)
    return rdf

In [40]:
stock_df = index_date(log_df)
print(stock_df.head())

            close  log_close
date                        
2015-11-24  37.59       3.63
2015-11-25  37.59       3.63
2015-11-26  37.82       3.63
2015-11-27  37.68       3.63
2015-11-28  37.68       3.63
